In [6]:
#Set Up
WKDIR=/workspace/hraijc/HiC_trials/HiC23/NovaSeq/SB1031

mkdir -p $WKDIR/log
cd $WKDIR

In [ ]:
/workspace/hraijc/HiC_trials/HiC23/Assemblies/SB1031

In [4]:
#IN INTERACTIVE SESSION:
#Use Srun not Salloc
"""
srun --time 01:00:00 --cpus-per-task 1 --mem-per-cpu 4G --pty bash
cd $WKDIR
git clone git@github.com:ignacio3437/hic-scaffolding-pipeline.git
cd ${WKDIR}/hic-scaffolding-pipeline/
git checkout dev
cd ${WKDIR}/hic-scaffolding-pipeline/containers
sudo -E singularity build pipeline_container.sif pipeline_container.def

"""

In [1]:
# RAW Hi-C Data
ls /input/genomic/plant/Actinidia/chinensis/ExperimentRequestor11022/AGRF_CAGRF230414403_HFFHYDSX7/Grape_1031_HFFHYDSX7*

/input/genomic/plant/Actinidia/chinensis/ExperimentRequestor11022/AGRF_CAGRF230414403_HFFHYDSX7/Grape_1031_HFFHYDSX7_CACGAT_L002_R1.fastq.gz
/input/genomic/plant/Actinidia/chinensis/ExperimentRequestor11022/AGRF_CAGRF230414403_HFFHYDSX7/Grape_1031_HFFHYDSX7_CACGAT_L002_R2.fastq.gz
/input/genomic/plant/Actinidia/chinensis/ExperimentRequestor11022/AGRF_CAGRF230414403_HFFHYDSX7/Grape_1031_HFFHYDSX7_CACGAT_L003_R1.fastq.gz
/input/genomic/plant/Actinidia/chinensis/ExperimentRequestor11022/AGRF_CAGRF230414403_HFFHYDSX7/Grape_1031_HFFHYDSX7_CACGAT_L003_R2.fastq.gz


In [3]:
#Combine Reads from flowcells

sbatch << EOF
#!/bin/bash
#SBATCH -J cat
#SBATCH --cpus-per-task=1
#SBATCH --mem=1G
#SBATCH --time=1:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err



cd /input/genomic/plant/Actinidia/chinensis/ExperimentRequestor11022/AGRF_CAGRF230414403_HFFHYDSX7
date
cat Grape_1031_HFFHYDSX7_CACGAT_L002_R1.fastq.gz Grape_1031_HFFHYDSX7_CACGAT_L003_R1.fastq.gz > /workspace/hraijc/TEMP/SB1031_HiC23_R1.fastq.gz
cat Grape_1031_HFFHYDSX7_CACGAT_L002_R2.fastq.gz Grape_1031_HFFHYDSX7_CACGAT_L003_R2.fastq.gz > /workspace/hraijc/TEMP/SB1031_HiC23_R2.fastq.gz
date




EOF


Submitted batch job 1948299


In [5]:
# Modify Config file

cat ${WKDIR}/hic-scaffolding-pipeline/nextflow.config

includeConfig './conf/base.config'

params {

    // R1, R2 dataset
    reads_R1                = "/workspace/hraijc/TEMP/SB1031_HiC23_R1.fastq.gz"
    reads_R2                = "/workspace/hraijc/TEMP/SB1031_HiC23_R2.fastq.gz"

    // Assembly file
    assembly_fasta          = "/workspace/hraijc/HiC_trials/HiC23/Assemblies/SB1031/SB1031_duplex_Q20_50kb_plushic.hic.purged.fa" // SALSA requires fasta to not contain ":" character

    // Prefix for output files
    out_prefix              = "SB1031_hic1"

    //STAG 1 
    stop_at_hicqc           = false

    //STAG 2: Include one specific scaffolder, report included
    only_run_yahs           = true
    only_run_salsa2         = false
    only_run_allhic         = false
    only_run_3D_DNA         = false
    

    /* https://github.com/c-zhou/yahs
    yahs_params{
        scaffolding_with_agp_file       = "" // for exmple: -a
        resolution                      = "" // for exmple: -r
        enzyme                          = "" /

In [8]:
# Run

sbatch << EOF
#!/bin/bash -e

#SBATCH -J hic_pipeline_${USER}
#SBATCH --time=167:00:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --mem=4G

ml nextflow/22.10.4
ml apptainer/1.1

cd ${WKDIR}/hic-scaffolding-pipeline

srun nextflow main.nf -resume
EOF





Submitted batch job 1948302
